In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly_resampler import register_plotly_resampler, FigureResampler, FigureWidgetResampler
from plotly.subplots import make_subplots

register_plotly_resampler(mode='auto')

In [15]:
force = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/JBZQ04/EXP2/JBZQ04-exp2-2023-07-21-kn.csv', header=1, names=['time', 'x_force', 'y_force'], usecols=[0,1,2])

displacement = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/JBZQ04/EXP2/JBZQ04-exp2-2023-07-21-mm.csv', header=1, names=['time', 'x_disp', 'y_disp', 'y_motor_disp', 'x_motor_disp'], usecols=[0,1,2,3,4])


force_2 = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/SMOOTH/LY-TEST-2023-06-13-kn.csv', header=1, names=['time', 'x_force', 'y_force'], usecols=[0,1,2])

displacement_2 = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/SMOOTH/LY-TEST-2023-06-13-mm.csv', header=1, names=['time', 'x_disp', 'y_disp', 'y_motor_disp', 'x_motor_disp'], usecols=[0,1,2,3,4])


force_sm1 = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/JULIASM1/SM1_force.csv', header=1, names=['time', 'x_motor_disp', 'x_disp', 'friction'], usecols=[0,1,2,3])

displacement_sm1 = pd.read_csv('C:/Users/lekim/OneDrive/Desktop/School/research-2023/mcgill-natural-faults/surfaces/JULIASM1/SM1_displacement.csv', header=1, names=['time', 'x_motor_disp', 'x_disp', 'y_motor_disp', 'y_disp'], usecols=[0,1,2,3,4])

In [16]:
# convert distances in sm1 from microns to mm
force_sm1['x_motor_disp'] = force_sm1['x_motor_disp'] / 1000
force_sm1['x_disp'] = force_sm1['x_disp'] / 1000

displacement_sm1['x_motor_disp'] = displacement_sm1['x_motor_disp'] / 1000
displacement_sm1['x_disp'] = displacement_sm1['x_disp'] / 1000
displacement_sm1['y_disp'] = displacement_sm1['y_disp'] / 1000
displacement_sm1['y_motor_disp'] = displacement_sm1['y_motor_disp'] / 1000

In [17]:
# define friction coefficient as ratio of shear force to normal force
force['friction'] = -force['x_force'] / force['y_force']


force_2['friction'] = -force_2['x_force'] / force_2['y_force']

# cut off data where friction is negative
displacement = displacement[force['friction'] > 0]
force = force[force['friction'] > 0]

displacement = displacement[force['friction'] < 0.9]
force = force[force['friction'] < 0.9]


displacement_2 = displacement_2[force_2['friction'] > 0]
force_2 = force_2[force_2['friction'] > 0]

displacement_2 = displacement_2[force_2['friction'] < 0.9]
force_2 = force_2[force_2['friction'] < 0.9]

# # define speed as the derivative of displacement
displacement['x_speed'] = np.gradient(displacement['x_disp'], displacement['time'])


displacement_2['x_speed'] = np.gradient(displacement_2['x_disp'], displacement_2['time'])


displacement_sm1['x_speed'] = np.gradient(displacement_sm1['x_disp'], displacement_sm1['time'])

# # smooth speed data
displacement['x_speed'] = displacement['x_speed'].rolling(50).mean()


displacement_2['x_speed'] = displacement_2['x_speed'].rolling(50).mean()


displacement_sm1['x_speed'] = displacement_sm1['x_speed'].rolling(50).mean()

# time_cutoff = 2438

# # cut off time data before 2438 seconds
# force = force[force['time'] > time_cutoff]
# displacement = displacement[displacement['time'] > time_cutoff]

# # shift time data to start at 0
# force['time'] = force['time'] - time_cutoff
# displacement['time'] = displacement['time'] - time_cutoff

# # shift displacement data up by 0.52 mm
# #displacement['x_disp'] = displacement['x_disp'] + 0.52

# # print last row of displacement data
# print(displacement.tail(1))

# add x_disp to force dataframe
force['x_disp'] = displacement['x_disp']
force['x_motor_disp'] = displacement['x_motor_disp']


force_2['x_disp'] = displacement_2['x_disp']
force_2['x_motor_disp'] = displacement_2['x_motor_disp']

# sort by column
sort_value = 'x_motor_disp'

force = force.sort_values(by=[sort_value])
displacement = displacement.sort_values(by=[sort_value])

force_2 = force_2.sort_values(by=[sort_value])
displacement_2 = displacement_2.sort_values(by=[sort_value])

force_sm1 = force_sm1.sort_values(by=[sort_value])
displacement_sm1 = displacement_sm1.sort_values(by=[sort_value])


C:\Users\lekim\AppData\Local\Temp\ipykernel_27668\761287132.py:15: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\lekim\AppData\Local\Temp\ipykernel_27668\761287132.py:18: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [18]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=force['x_motor_disp'], y=force['friction'], name='friction', connectgaps=True, mode='lines+markers'), secondary_y=False)

fig.add_trace(go.Scatter(x=displacement['x_motor_disp'], y=displacement['x_speed'], name='speed', connectgaps=True, mode='lines+markers'), secondary_y=True)


fig.add_trace(go.Scatter(x=force_2['x_motor_disp'], y=force_2['friction'], name='friction', connectgaps=True, mode='lines+markers'), secondary_y=False)

fig.add_trace(go.Scatter(x=displacement_2['x_motor_disp'], y=displacement_2['x_speed'], name='speed', connectgaps=True, mode='lines+markers'), secondary_y=True)


fig.add_trace(go.Scatter(x=force_sm1['x_motor_disp'], y=force_sm1['friction'], name='friction', connectgaps=True, mode='lines+markers'), secondary_y=False)

fig.add_trace(go.Scatter(x=displacement_sm1['x_motor_disp'], y=displacement_sm1['x_speed'], name='speed', connectgaps=True, mode='lines+markers'), secondary_y=True)

# Add steps as vertical lines
x_steps = np.array([2, 3, 4, 5, 8, 13])

t_steps = np.array([0, 100, 180, 230, 33, 360, 460, 520, 620, 740, 840, 1140, 1240, 1840, 2840, 3040, 3200, 3300, 3400, 3450, 3550, 3580, 3680, 3740, 3840, 3960, 4060, 4360, 4460, 5060, 5560])

# multiply by 50 to get time step
t_steps = t_steps + 162

fig.update_layout(width=1120, height=600, title='Friction vs. Displacement', xaxis_title='Displacement (mm)', yaxis_title='Friction Coefficient', font=dict(size=12))

fig


FigureWidgetResampler({
    'data': [{'connectgaps': True,
              'mode': 'lines+markers',
              'name': '<b style="color:sandybrown">[R]</b> friction <i style="color:#fc9944">~0.08</i>',
              'type': 'scatter',
              'uid': 'f13e11ac-039c-42a1-9d8f-2d6440e2da1d',
              'x': array([ 0.        ,  0.        ,  0.1126565 , ..., 74.9003049 , 74.99999739,
                          75.00000484]),
              'xaxis': 'x',
              'y': array([8.97694165e-01, 4.43220463e-06, 1.63402863e-02, ..., 7.69788609e-01,
                          8.99935686e-01, 8.99973886e-01]),
              'yaxis': 'y'},
             {'connectgaps': True,
              'mode': 'lines+markers',
              'name': '<b style="color:sandybrown">[R]</b> speed <i style="color:#fc9944">~0.08</i>',
              'type': 'scatter',
              'uid': '1a3e08dd-9d0f-4c66-9467-5236fd059956',
              'x': array([ 0.        ,  0.        ,  0.104798  , ..., 74.84730519, 7